In [1]:
from tool import processor, correlation, resampling

### Rank Clustering

In [2]:
path = './data/segment_level/raw'
f = processor.FileProcessor(path)

In [3]:
df = f.makeFrame()
df.head(3)

,sacreTer.Hannanum,sacreTer.Khaiii,sacreTer.Kiwi,sacreTer.Kkma,sacreTer.Mecab,sacreTer.Komoran,sacreTer.None,sacreTer.Okt,sacreTer.Spm,sacreBleu.Jamo,...,sacreChrf.Kkma,sacreChrf.Komoran,sacreChrf.Mecab,sacreChrf.None,sacreChrf.Okt,sacreChrf.Spm,sacreChrf.Character,sacreTer.Jamo,sacreTer.Character,Human
0,64.7059,50.0,50.0000,47.8261,50.0000,50.0000,77.7778,66.6667,50.0000,53.4154,...,53.3701,53.0081,52.3972,50.6958,48.1593,52.3972,54.0168,34.8485,44.8276,-1.0107
1,56.5217,62.5,54.1667,50.0000,60.0000,54.1667,66.6667,55.5556,59.0909,40.8603,...,44.6038,44.7165,41.2800,39.2493,41.2018,41.4562,41.8010,47.0588,52.9412,-1.0107
2,27.5862,25.0,25.0000,29.0323,25.8065,22.5806,45.4545,33.3333,25.8065,60.0980,...,66.3012,70.0629,68.7973,65.4657,67.0995,68.7973,67.6693,24.4444,25.6410,1.1051


In [4]:
for metric in ['bleu', 'chrf', 'ter']:
    pearson = correlation.Correlation(f.selectFrame(metric))
    print(pearson.rank_cluster(), '\n')

[('sacreBleu.Character', 0), ('sacreBleu.Kiwi', 0), ('sacreBleu.Komoran', 0), ('sacreBleu.Jamo', 0), ('sacreBleu.Khaiii', 1), ('sacreBleu.Kkma', 2), ('sacreBleu.Mecab', 2), ('sacreBleu.Spm', 3), ('sacreBleu.Okt', 4), ('sacreBleu.Hannanum', 5), ('sacreBleu.None', 6)] 

[('sacreChrf.Komoran', 0), ('sacreChrf.Khaiii', 0), ('sacreChrf.Kiwi', 1), ('sacreChrf.Kkma', 2), ('sacreChrf.Character', 2), ('sacreChrf.Mecab', 2), ('sacreChrf.Spm', 2), ('sacreChrf.Okt', 3), ('sacreChrf.Hannanum', 4), ('sacreChrf.None', 5), ('sacreChrf.Jamo', 6)] 

[('sacreTer.None', 0), ('sacreTer.Hannanum', 1), ('sacreTer.Jamo', 1), ('sacreTer.Okt', 2), ('sacreTer.Character', 3), ('sacreTer.Khaiii', 4), ('sacreTer.Spm', 4), ('sacreTer.Komoran', 4), ('sacreTer.Mecab', 5), ('sacreTer.Kiwi', 6), ('sacreTer.Kkma', 6)] 



### Bootstrap Resampling

In [ ]:
# i.e = BLEU
samples = resampling.Resampling(frame=f.selectFrame('bleu'),
                    m=3000, n=1000, iteration=10)
samples.bootstrap_resampling(2)

### Automatic Metric Scoring 

#### SacreBLEU

In [7]:
from tool.metric.sacreBLEU import SacreBLEU

In [8]:
hypothesis = "모델 레온 데임(20)은 아무도 밟지 않은 것처럼 캣워크를 활보하며 한 발을 다른 발 앞에 강하게 내딛고 활주로를 밟으며 강렬하고 강철 같은 섬광을 그대로 유지했다."
reference = ["모델 레옹 데임(20)은 아직 그 누구도 시도한 적 없는 방식으로 캣워크를 활보했다. 그는 강렬하고 찌를 듯한 눈빛을 유지한 채 한 발 한 발 힘차게 내뻗으며 런웨이를 밟아 나갔다."]

In [9]:
metric = SacreBLEU('bleu')

metric.hypothesis=hypothesis
metric.references=reference

In [10]:
metric.get_score(sys_level=False)

BLEU = 12.35 37.9/14.3/11.1/7.7 (BP = 0.842 ratio = 0.853 hyp_len = 29 ref_len = 34)

#### NLTK